In [1]:
!pip install pandas folium numpy
import pandas as pd
import folium
import random
import math
import numpy as np


In [2]:
# Load customer data
df = pd.read_csv(r'C:\Users\Samuel\Documents\customers_gwagwalada.csv')

# Extract locations and demands
depot = (df.iloc[0]['latitude'], df.iloc[0]['longitude'])
customers = df.iloc[1:].to_dict('records')
locations = [depot] + [(customer['latitude'], customer['longitude']) for customer in customers]
demands = [customer['demand_bags'] + customer['demand_packs'] for customer in customers]
print(df.head())

   id  latitude  longitude  demand_bags  demand_packs
0   0  8.942050   7.081990            0             0
1   1  8.925081   7.052678            8             3
2   2  8.900276   7.067616           16             3
3   3  8.903489   7.050503            3             4
4   4  8.940240   7.094881           14             1


In [3]:
# Calculate distance matrix (Haversine distance)
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

distance_matrix = [[haversine_distance(*loc1, *loc2) for loc2 in locations] for loc1 in locations]

In [4]:
# Parameters
num_vehicles = 5
capacity = 220

# Initial solution generation
def get_initial_solution(demands, capacity):
    routes = []
    remaining_customers = set(range(1, len(demands) + 1))  # Start from 1 since 0 is depot
    while remaining_customers:
        route = []
        load = 0
        while remaining_customers:
            customer = remaining_customers.pop()
            if load + demands[customer - 1] <= capacity:
                route.append(customer)
                load += demands[customer - 1]
            else:
                remaining_customers.add(customer)
                break
        routes.append(route)
    return routes

initial_solution = get_initial_solution(demands, capacity)

In [5]:
# Function to calculate total distance of a solution
def total_distance(solution, distance_matrix):
    total = 0
    for route in solution:
        route = [0] + route + [0]  # Add depot to start and end
        for i in range(len(route) - 1):
            total += distance_matrix[route[i]][route[i + 1]]
    return total

In [6]:
# Function to destroy part of the solution
def destroy_solution(solution, num_customers):
    removed_customers = []
    for _ in range(num_customers):
        route = random.choice(solution)
        if route:
            customer = route.pop(random.randint(0, len(route) - 1))
            removed_customers.append(customer)
    return removed_customers

# Function to insert removed customers back into the solution
def insert_customers(solution, removed_customers, distance_matrix):
    for customer in removed_customers:
        best_increase = float('inf')
        best_position = None
        best_route = None
        for route in solution:
            for i in range(len(route) + 1):
                new_route = route[:i] + [customer] + route[i:]
                new_distance = 0
                for j in range(len(new_route) - 1):
                    new_distance += distance_matrix[new_route[j]][new_route[j + 1]]
                new_distance += distance_matrix[0][new_route[0]] + distance_matrix[new_route[-1]][0]  # Depot to first and last
                increase = new_distance - total_distance([route], distance_matrix)
                if increase < best_increase:
                    best_increase = increase
                    best_position = i
                    best_route = route
        best_route.insert(best_position, customer)
    return solution

In [7]:
# Large Neighborhood Search (LNS) algorithm
def large_neighborhood_search(initial_solution, distance_matrix, num_iterations=100, destroy_rate=0.2):
    best_solution = initial_solution
    best_cost = total_distance(best_solution, distance_matrix)
    
    for _ in range(num_iterations):
        current_solution = [route[:] for route in best_solution]  # Deep copy of the solution
        num_customers_to_remove = int(destroy_rate * sum(len(route) for route in current_solution))
        removed_customers = destroy_solution(current_solution, num_customers_to_remove)
        current_solution = insert_customers(current_solution, removed_customers, distance_matrix)
        
        current_cost = total_distance(current_solution, distance_matrix)
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost
    
    return best_solution

In [8]:
# Run the LNS algorithm
best_solution = large_neighborhood_search(initial_solution, distance_matrix)

# Print Best Solution
print("Best solution found:", best_solution)
print("Total distance:", total_distance(best_solution, distance_matrix))

Best solution found: [[70, 85, 81, 95, 18, 10, 35, 43, 80, 47, 32, 39], [76, 37, 44, 71, 91, 27, 19, 17, 38, 69, 21, 74, 29, 8, 55, 79, 5, 97, 83, 53, 58, 7, 63, 12, 22, 24, 61, 59, 54, 14, 46, 4, 82, 94, 88, 11, 99, 60, 93, 42, 87], [], [], [], [96, 33, 34, 77, 31, 45, 9, 28, 20, 30, 62, 89, 16, 72, 15, 100, 86, 1, 40, 51, 73, 64, 68, 25, 78, 66, 36, 23, 6, 2, 92, 98, 3, 50, 13, 49, 67, 52, 90, 57, 48, 56, 26, 75, 65, 41, 84], []]
Total distance: 59.4067136429702


In [9]:
# Visualize with Folium

# Create Folium Map
map_center = depot
vrp_map = folium.Map(location=map_center, zoom_start=14)

# Add Depot Marker
folium.Marker(
    location=depot,
    popup="Depot",
    icon=folium.Icon(color='red', icon='industry', prefix='fa'),
).add_to(vrp_map)

# Add Customer Markers with Popups
for idx, (lat, lon) in enumerate(locations[1:]):
    demand = demands[idx]
    folium.Marker(
        location=[lat, lon],
        popup=f"Customer {idx + 1}, Demand: {demand}",
    ).add_to(vrp_map)

colors = ['blue', 'green', 'purple', 'orange', 'red']  # Add more colors if needed
for vehicle_id, route in enumerate(best_solution):
    route_with_depot = [0] + route + [0]  # Add depot to start and end
    route_coordinates = [locations[i] for i in route_with_depot]
    folium.PolyLine(route_coordinates, color=colors[vehicle_id % len(colors)], weight=2.5, opacity=1).add_to(vrp_map)

# Show the map
vrp_map

In [10]:
def run_experiment():
    customers = load_customers('customers_gwagwalada.csv')
    num_vehicles = 5
    capacity = 1000
    algorithm = 'LNS'  # Example algorithm

    if algorithm == 'LNS':
        solution = large_neighborhood_search(customers, num_vehicles, capacity)
    elif algorithm == 'Tabu':
        solution = tabu_search(customers, num_vehicles, capacity)
    elif algorithm == 'ILP':
        solution = ilp_solver(customers, num_vehicles, capacity)
    
    visualize_routes(solution)
    print('Experiment completed.')
